In [53]:
import pandas as pd
import heapq
from functools import total_ordering

In [54]:
clients = pd.read_csv("DataSets\example-set\input_clients.csv")
display(clients)

<>:1: SyntaxWarning: invalid escape sequence '\e'
<>:1: SyntaxWarning: invalid escape sequence '\e'
C:\Users\reane\AppData\Local\Temp\ipykernel_11588\439770960.py:1: SyntaxWarning: invalid escape sequence '\e'
  clients = pd.read_csv("DataSets\example-set\input_clients.csv")


,ClientID,Currencies,PositionCheck,Rating
0,A,"USD,SGD",Y,1
1,B,"USD,SGD,JPY",N,2
2,C,SGD,Y,3
3,D,USD,Y,4
4,E,SGD,N,5


In [55]:
instruments = pd.read_csv("DataSets\example-set\input_instruments.csv")
display(instruments)

<>:1: SyntaxWarning: invalid escape sequence '\e'
<>:1: SyntaxWarning: invalid escape sequence '\e'
C:\Users\reane\AppData\Local\Temp\ipykernel_11588\4001465702.py:1: SyntaxWarning: invalid escape sequence '\e'
  instruments = pd.read_csv("DataSets\example-set\input_instruments.csv")


,InstrumentID,Currency,LotSize
0,SIA,SGD,100


In [56]:
orders = pd.read_csv("DataSets\example-set\input_orders.csv")
display(orders)

<>:1: SyntaxWarning: invalid escape sequence '\e'
<>:1: SyntaxWarning: invalid escape sequence '\e'
C:\Users\reane\AppData\Local\Temp\ipykernel_11588\3954892461.py:1: SyntaxWarning: invalid escape sequence '\e'
  orders = pd.read_csv("DataSets\example-set\input_orders.csv")


,Time,OrderID,Instrument,Quantity,Client,Price,Side
0,9:00:01,A1,SIA,1500,A,Market,Buy
1,9:02:00,B1,SIA,4500,B,32.1,Sell
2,9:05:00,C1,SIA,100,C,32,Buy
3,9:10:00,D1,SIA,300,D,Market,Sell
4,9:29:01,B2,SIA,5,B,32.1,Sell
5,9:29:02,E1,SIA,1000,E,32,Sell
6,9:29:03,A2,SIA,800,A,31.9,Buy
7,9:30:01,C2,SIA,100,C,Market,Sell
8,9:40:00,B3,SIA,500,B,32.2,Sell
9,10:50:00,C3,SIA,4200,C,32.2,Buy


In [57]:
class Client:
    def __init__(self, ID, Currencies, PosCheck, Rating):
        self.ID = ID
        self.Currencies = Currencies
        self.PosCheck = PosCheck
        self.Rating = Rating
        self.NetPosition = 0

In [58]:
@total_ordering
class Order:
    def __init__(self, Time, orderID, Instrument, Quantity, Client, Price, Side):
        self.Time =Time
        self.orderID = orderID
        self.Instrument = Instrument
        self.Quantity = Quantity
        self.Client = Client
        self.Price = Price
        self.Side = Side
        self.Rejection = False
        self.RejectionRsn = None
        self.ClientRating = None

    
    def __lt__(self, other):
        
        
        if (self.Price != other.Price):
            # Check for -1 for market order
            if (self.Price == -1): return True
            elif (other.Price == -1): return False
            if (self.Price < other.Price & self.Side == "Sell"):
                return True
            elif (self.Price > other.Price & self.Side == "Sell"):
                return False
            elif (self.Price > other.Price & self.Side == "Buy"):
                return True
            elif (self.Price < other.Price & self.Side == "Buy"):
                return False

        if (self.ClientRating > other.ClientRating):
            return True
        elif (self.ClientRating < other.ClientRating):
            return False
        elif (self.Time < other.Time):
            return True
        elif (self.Time >= other.Time):
            return False

    # def __gt__(self, other):
    #     if (self.Price > other.Price & self.Side == "Sell"):
    #         return True
    #     elif (self.Price < other.Price & self.Side == "Sell"):
    #         return False
    #     elif (self.Price < other.Price & self.Side == "Buy"):
    #         return True
    #     elif (self.Price > other.Price & self.Side == "Buy"):
    #         return False
    #     elif (self.ClientRating < other.ClientRating):
    #         return True
    #     elif (self.ClientRating > other.ClientRating):
    #         return False
    #     elif (self.Time > other.Time):
    #         return True
    #     elif (self.Time <= other.Time):
    #         return False

    def CheckOrder(self, Client, InstrumentNameList, InstrumentList):
        
        if (self.Instrument not in InstrumentNameList):
            #InstrumentNameList is just the InstrumentID column
            self.Rejection = True
            self.RejectionRsn = "REJECTED-INSTRUMENT NOT FOUND"
            return False
        else:
            for x in InstrumentList:
                if (x.ID == self.Instrument):
                    Instrument = x
        if (Instrument.Currency not in Client.Currencies):
            #REJECT MISMATCH CURRENCY
            self.Rejection = True
            self.RejectionRsn = "REJECTED-MISMATCH CURRENCY"
            return False
        elif (self.Quantity % Instrument.LotSize):
            self.Rejection = True
            self.RejectionRsn = "REJECTED-INVALID LOT SIZE"
            return False
        elif ((Client.PosCheck == "Y") & 
              (self.Side == "Sell") & 
              ((Client.NetPosition - self.Quantity) < 0)):
            self.Rejection = True
            self.RejectionRsn = "REJECTED-POSITION CHECK FAILED" + str(Client.NetPosition)
            return False
        
        self.ClientRating = Client.Rating

        return True

In [59]:
class Instrument:
    def __init__(self, ID, Currency, LotSize):
        self.ID = ID
        self.Currency = Currency
        self.LotSize = LotSize
        self.OpenPrice = None
        self.ClosePrice = None
        self.TotalVolume = 0
        self.PricexVol = 0
        self.DayHigh = None
        self.DayLow = None
        self.sellheap = []
        self.buyheap = []
    def heappush(self, NewOrder):
        if (Order.Side == "Sell"):
            heapq.heappush(self.sellheap, NewOrder)
        if (Order.Side == "Buy"):
            heapq.heappush(self.buyheap, NewOrder)
    def heapread(self, side):
        if (side == "Buy"):
            return self.buyheap[0]
        else:
            return self.sellheap[0]
    def heappop(self, side):
        if (side == "Buy"):
            return heapq.heappop(self.buyheap)
        else:
            return heapq.heappop(self.sellheap)



In [60]:
rejection = pd.DataFrame(columns = ['Order Id', 'Rejection Reason'])

In [61]:
def createClientList(clients):
    clientList = []
    clientHash = {}
    for i in range(len(clients.index)):
        row = clients.iloc[i]
        client = Client(row['ClientID'], row['Currencies'], row['PositionCheck'], row['Rating'])
        clientList.append(client)
        clientHash[(row['ClientID'])] = client
    return clientList, clientHash

def findClient(clientId, clients):
    for client in clients:
        if client.ID == clientId:
            return client

In [62]:
def createInstrumentList(instruments):
    instrumentList = []
    instrumentNameList = []
    instrumentHash = {}
    for i in range(len(instruments.index)):
        row = instruments.iloc[i]
        instrument = Instrument(row['InstrumentID'], row['Currency'], row['LotSize'])
        instrumentList.append(instrument)
        instrumentNameList.append(row['InstrumentID'])
        instrumentHash[row["InstrumentID"]] = instrument
    return instrumentList, instrumentNameList, instrumentHash

In [63]:
clientList, clientHash = createClientList(clients)
instrumentList, instrumentNameList, tickerHash = createInstrumentList(instruments)
openOrderList = []
continuousOrderList = []
closeOrderList = []

In [64]:
def buildOrderBook(openOrderList, continuousOrderList, closeOrderList, clients, instruments, orders, rejection):
    for i in range(len(clients.index)):
        row = orders.iloc[i]
    
    orders['Time'] = pd.to_datetime(orders['Time'], format="%H:%M:%S")

    for i in range(1,len(orders.index)):
        row = orders.iloc[i]
        price = row['Price']
        if row['Price'] == 'Market':
            price = -1
        currClient = findClient(row['Client'], clientList)
        order = Order(row['Time'], row['OrderID'], row['Instrument'], row['Quantity'], currClient, price, row['Side'])
        if order.CheckOrder(currClient, instrumentNameList, instrumentList) == False:
                rejection.loc[len(rejection.index)] = (order.orderID, order.RejectionRsn)
        else:
            if row['Time'] < pd.to_datetime('9:30:00', format="%H:%M:%S"):
                if order.CheckOrder(currClient, instrumentNameList, instrumentList) == False:
                    rejection.loc[len(rejection.index)] = (order.orderID, order.RejectionRsn)
                else:
                    openOrderList.append(order)
            elif row['Time'] < pd.to_datetime('16:00:00', format="%H:%M:%S"):
                 continuousOrderList.append(order)
            else:
                 closeOrderList.append(order)
        # if row['Time'] < pd.to_datetime('9:30:00', format="%H:%M:%S"):
        #     if order.CheckOrder(currClient, instrumentNameList, instrumentList) == False:
        #         rejection.loc[len(rejection.index)] = (order.orderID, order.RejectionRsn)
        #     else:
        #         orderList.append(order)
        # else:
        #     if order.CheckOrder(currClient, instrumentNameList, instrumentList) == False:
        #         rejection.loc[len(rejection.index)] = (order.orderID, order.RejectionRsn)
            
rejection = pd.DataFrame(columns = ['OrderId', 'Rejection Reason'])
buildOrderBook(openOrderList, continuousOrderList, closeOrderList, clients, instruments, orders, rejection)
rejection.to_csv('output_exchange_report.csv')

in()


In [65]:
buildOrderBook()
def main():
    tickerHash = {}
    with open('./input_instruments.csv', mode = 'r') as file:
        csvFile = csv.reader(file)
        for lines in csvFile:
            if (lines[0]) == "InstrumentID":
                continue
            tickerHash[lines[0]] = Instrument(lines[0], lines[1], lines[2])
    # with open('./input_orders.csv', mode = 'r')as file:
    #     csvFile = csv.reader(file)
    #     for lines in csvFile:
    #         if (lines[0]) == "Time":
    #             continue
    #         orderLst.append(lines)
    for order in orderList:
        ticker = tickerHash[order[2]]
        exchangeType = order[6]##this 
        if (exchangeType == "Buy"):
            #ticker.heapPush(order)
            pass
        else:
            #ticker.heapPush(order)
            pass
        bestBuyOrder = ticker.heapPop()
        bestSellOrder = ticker.heapPop()
        bestBuyOrderPrice = bestBuyOrder.price
        bestSellOrderPrice = bestSellOrder.price
        '''
        if (bestBuyOrderPrice >= bestSellOrderPrice):
            maxVol = max(bestBuyOrder.volume, bestSellOrder.volume)
            bestBuyOrder.volume = bestBuyOrder.volume - maxVol 
            bestSellOrder.volume = bestSellOrder.volume - maxVol 
            if (bestBuyOrder.volume == 0):
                ticker.sellPQ.insert(bestBuyOrder)
            else:
                ticer.buyPq.insert(bestSellOrder)
        '''

main()


TypeError: buildOrderBook() missing 7 required positional arguments: 'openOrderList', 'continuousOrderList', 'closeOrderList', 'clients', 'instruments', 'orders', and 'rejection'

In [ ]:
#Function to Match at 09:30

def Auction(Instrument):
    matching = True
    while (matching):
        
        if (Instrument.heapread('Buy')[-1].Price == -1):
            if (Instrument.heapread('Sell')[-1].Price == -1):
                

        Instrument.heapread('Buy')[0]
        